<a href="https://colab.research.google.com/github/vitaldb/plans/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PlanS Model for predicting hepatocellular carcinoma in hepatitis B patients

In [2]:
!pip install scikit-survival
!wget https://github.com/vitaldb/plans/raw/main/model.pkl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 5.3 MB/s 
--2022-07-22 04:52:10--  https://github.com/vitaldb/plans/raw/main/model.pkl
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/plans/main/model.pkl [following]
--2022-07-22 04:52:10--  https://raw.githubusercontent.com/vitaldb/plans/main/model.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1409741 (1.3M) [application/octet-stream]
Saving to: ‘model.pkl’

model.pkl           100%[===================>]   1.34M  --.-KB/s    in 0.05s  

In [11]:
from sksurv.ensemble import RandomSurvivalForest
import numpy as np
import sys
import pickle

# load model
model = pickle.load(open('model.pkl', 'rb'))

# sample values
male = 0
e_t = 1  # entecavir (ETV) -> 0, tenofovir (TDF) -> 1
lc = 1  # None for missing value
plt = 172
tbil = 1.1
alt = 122
dna = 74279943
hbeag = 1

x = np.array([male, e_t, lc, plt, tbil, alt, dna, hbeag]).astype(float)
x = x[None, ...]

y_pred = model.predict(x).flatten()[0]
y_harz = model.predict_cumulative_hazard_function(x, return_array=True).flatten()

x[:, 1] = 0
y_pred_e = model.predict(x).flatten()[0]

x[:, 1] = 1
y_pred_t = model.predict(x).flatten()[0]

In [12]:
print(f'risk score = {y_pred:.3f}')
print(f'risk score (entecavir) = {y_pred_e:.3f}')
print(f'risk score (tenofovir) = {y_pred_t:.3f}')
print(model.event_times_)
print(y_harz)

risk score = 5.556
risk score (entecavir) = 5.832
risk score (tenofovir) = 5.556
[ 12.  14.  15.  16.  18.  19.  20.  21.  22.  23.  24.  25.  26.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  40.  42.  43.  44.  45.
  47.  48.  49.  51.  52.  53.  54.  55.  56.  58.  59.  60.  61.  63.
  64.  65.  67.  69.  70.  71.  73.  74.  75.  76.  77.  78.  79.  82.
  83.  84.  85.  87.  88.  91.  92.  94.  97. 113. 114. 118. 127. 131.
 133.]
[0.00039647 0.00188133 0.00339892 0.00430027 0.0084794  0.00861007
 0.01330625 0.01379799 0.01418202 0.01794592 0.01811267 0.0196442
 0.02000561 0.02264777 0.02355478 0.02463591 0.02986037 0.03420457
 0.03731337 0.03962576 0.03980364 0.04052809 0.04474592 0.0463422
 0.04656117 0.04881494 0.0498603  0.05075924 0.05272361 0.05626422
 0.05797336 0.06081432 0.06110282 0.06159321 0.06195993 0.06307524
 0.0631591  0.07021243 0.07156848 0.07178372 0.07358847 0.08243208
 0.08577    0.08758538 0.089519   0.09496836 0.09882857 0.1064198
 0.11008655 0.11185033 0